In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
try:
    df = pd.read_csv('food_nutrient_temperament.csv')
except FileNotFoundError:
    print("Error: File 'food_nutrient_pivot.csv' not found. Please check the file path.")
    exit()

# Columns handling
required_columns = ['food_ description', 'Temperament']
if not all(col in df.columns for col in required_columns):
    missing_cols = [col for col in required_columns if col not in df.columns]
    print(f"Error: Missing columns {missing_cols} in DataFrame.")
    exit()

# preprocessing
X = df.iloc[:, 2:-1].fillna(0)  # ویژگی‌های غذایی
y = df['Temperament']
feature_names = X.columns

# missing value handling in Temperament
if y.isnull().any():
    print("Warning: Missing values found in 'Temperament'. Filling with -1 (Unknown).")
    y = y.fillna(-1)
    df['Temperament'] = y

# make Temperament numerical
df['Temperament'] = pd.to_numeric(df['Temperament'], errors='coerce').fillna(-1).astype(int)

# Unique Temperament values
unique_temperaments = df['Temperament'].unique()
print("\nUnique Temperament Values:", unique_temperaments)

# Normalization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA with 5 components
pca = PCA(n_components=5)
pca_result = pca.fit_transform(X_scaled)
print("Explained Variance Ratio for 5 Components:", pca.explained_variance_ratio_)
print("Total Variance Explained:", sum(pca.explained_variance_ratio_))

# K-Means clustering
n_clusters = [3, 4]  # آزمایش با ۳ و ۴ خوشه
best_kmeans = None
best_score = -1
best_n_clusters = 0
silhouette_scores = []

for k in n_clusters:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(pca_result)
    score = silhouette_score(pca_result, cluster_labels)
    silhouette_scores.append(score)
    print(f"Silhouette Score for {k} clusters: {score:.3f}")
    if score > best_score:
        best_score = score
        best_kmeans = kmeans
        best_n_clusters = k

# final clustering with best clusters
cluster_labels = best_kmeans.fit_predict(pca_result)
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(5)], index=df.index)
pca_df['food_ description'] = df['food_ description']
pca_df['Temperament'] = df['Temperament']
pca_df['Cluster'] = cluster_labels

# Number of foods per cluster
print("\nNumber of Foods per Cluster:")
print(pca_df['Cluster'].value_counts())

# each food in clusters
for cluster in range(best_n_clusters):
    print(f"\nFoods in Cluster {cluster}:")
    print(pca_df[pca_df['Cluster'] == cluster][['food_ description', 'Temperament']])

# features mean
cluster_features = []
for cluster in range(best_n_clusters):
    cluster_mask = pca_df['Cluster'] == cluster
    features_mean = df[cluster_mask][feature_names].mean()
    cluster_features.append(features_mean)
    print(f"\nMean Features for Cluster {cluster} (Top 10):")
    print(features_mean.sort_values(ascending=False).head(10))

# visualization 1: bar chart for each cluster
plt.figure(figsize=(15, 5 * best_n_clusters))
for cluster in range(best_n_clusters):
    plt.subplot(best_n_clusters, 1, cluster + 1)
    cluster_features[cluster].plot(kind='bar', color='C' + str(cluster), alpha=0.6)
    plt.title(f'Mean Features for Cluster {cluster}')
    plt.ylabel('Mean Value')
    plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# vis 2: comparison bar charts
key_features = ['water (g)', 'energy (KCAL)', 'protein (g)', 'total_sugars (g)', 'potassium_k (mg)',
                'sodium_na (mg)', 'zinc_zn (mg)', 'cholesterol (mg)', 'vitamin_b-6 (mg)', 'total_lipid_fat (g)']
if not all(feat in feature_names for feat in key_features):
    print(f"Warning: Some key features {key_features} not found in DataFrame columns.")
else:
    feature_means = pd.DataFrame({
        f'Cluster {i}': cluster_features[i][key_features] for i in range(best_n_clusters)
    })
    feature_means.plot(kind='bar', figsize=(12, 6), alpha=0.6)
    plt.title('Comparison of Mean Key Features Across Clusters')
    plt.ylabel('Mean Value')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Cluster')
    plt.tight_layout()
    plt.show()

# vis3: PC1-PC2
plt.figure(figsize=(10, 8))
for cluster in range(best_n_clusters):
    mask = pca_df['Cluster'] == cluster
    plt.scatter(pca_df[mask]['PC1'], pca_df[mask]['PC2'], label=f'Cluster {cluster}', alpha=0.6)
plt.axvline(x=-2, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.axhline(y=-2, color='gray', linestyle='--', alpha=0.5)
plt.axhline(y=4, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('PC1 (34.07%)')
plt.ylabel('PC2 (15.63%)')
plt.title('Clustering of Foods in PCA Space (PC1 vs PC2)')
plt.legend()
plt.show()

# Valid Temperament filtering
valid_temperaments = [0, 1, 2]  # 0: Cold, 1: Hot, 2: Moderate
valid_mask = pca_df['Temperament'].isin(valid_temperaments)
temperament_labels = pca_df['Temperament'][valid_mask]
filtered_cluster_labels = cluster_labels[valid_mask]

# valid temperament values
if valid_mask.sum() == 0:
    print("Error: No valid Temperament values found (expected: 0, 1, 2). All values are:", unique_temperaments)
    print("Cannot create confusion matrix or train model.")
else:
    print("\nNumber of valid samples:", len(temperament_labels))
    print("Number of cluster labels:", len(filtered_cluster_labels))
    
    # بررسی مقادیر غیرمنتظره
    invalid_temperaments = pca_df[~valid_mask]['Temperament'].unique()
    if len(invalid_temperaments) > 0:
        print(f"Warning: Invalid Temperament values found: {invalid_temperaments}. Excluded from confusion matrix.")
    else:
        print("All Temperament values are valid.")
    

# Loadings
loadings = pd.DataFrame(pca.components_.T, columns=[f'PC{i+1}' for i in range(5)], index=feature_names)
print("\nComponent Loadings:\n", loadings)